In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams

rcParams['figure.figsize'] = 20,10 #sets figure size
from keras.models import Sequential #defines a linear stack of layers for deep learning models
from keras.layers import LSTM,Dropout,Dense 
# LSTM = great for sequential data like stock prices
# Dropout = tool used to help prevent overfitting by randomly dropping connections in the neural network
# Dense = a fully connected layer

from sklearn.preprocessing import MinMaxScaler #normalisation technique